### download the last month of the CDS raw forecasts 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
HOME = pathlib.Path.home()

In [4]:
CWD = pathlib.Path.cwd()

In [5]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [6]:
sys.path.append('../..')

In [7]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [8]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [140, 240, -50, 30],
 'Water_Watch': [120, 240, -38, 22.5]}

### provider (always CDS for now)

In [9]:
provider = 'CDS'

### variable name

In [10]:
var_name = 'tprate'

### list of GCMs 

In [11]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','DWD', 'CMCC', 'NCEP', 'JMA', 'ECCC']

### list of systems 

In [13]:
list_systems = [5, 600, 7, 21, 35, 2, 2, 2]

In [14]:
dict_systems = dict(zip(list_GCMs, list_systems))

In [15]:
dict_systems

{'ECMWF': 5,
 'UKMO': 600,
 'METEO_FRANCE': 7,
 'DWD': 21,
 'CMCC': 35,
 'NCEP': 2,
 'JMA': 2,
 'ECCC': 2}

### lag in months (if need to download older forecasts)

In [12]:
lag = 0 

### get today's date 

In [13]:
date = datetime.utcnow()

### apply lag 

In [14]:
date = date - relativedelta(months=lag)

In [15]:
print(f"will download forecasts initialised {date:%Y - %m}")

will download forecasts initialised 2021 - 06


### path to download the realtime forecasts 

In [17]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/{provider}/operational/forecasts')

In [18]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### Now loops over the list of GCM and downloads, for the parameters provided 

In [19]:
for GCM in list_GCMs: 
    C3S.download(
    GCM=GCM,
    system=dict_systems[GCM]
    varname=var_name,
    year=date.year,
    month=date.month,
    leadtimes=[1, 2, 3, 4, 5, 6],
    opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
    domain=domains.domains['C3S_download'],
    file_format='netcdf',
    level='surface',
    max_retry=3,
    )   


attempting to download tprate for GCM ECMWF, for level surface, year 2021, month 6, in netcdf, attempt 1 of 3



2021-06-14 09:45:51,918 INFO Welcome to the CDS
2021-06-14 09:45:51,920 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-06-14 09:45:52,237 INFO Request is queued
2021-06-14 09:47:48,574 INFO Request is running
2021-06-14 09:48:46,550 INFO Request is completed
2021-06-14 09:48:46,552 INFO Downloading https://download-0005.copernicus-climate.eu/cache-compute-0005/cache/data4/adaptor.mars.external-1623620863.572268-7229-19-c22dcba1-568f-46cc-a8b6-772747f1de41.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_ECMWF.netcdf (4M)
2021-06-14 09:49:00,642 INFO Download rate 289.4K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_ECMWF.netcdf downloaded OK


attempting to download tprate for GCM UKMO, for level surface, year 2021, month 6, in netcdf, attempt 1 of 3



2021-06-14 09:49:01,229 INFO Welcome to the CDS
2021-06-14 09:49:01,231 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-06-14 09:49:01,529 INFO Request is queued
2021-06-14 09:50:19,102 INFO Request is completed
2021-06-14 09:50:19,104 INFO Downloading https://download-0002.copernicus-climate.eu/cache-compute-0002/cache/data8/adaptor.mars.external-1623621006.0494678-1447-1-1b7f581d-ebf4-4135-af60-e2bc37381d2a.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/UKMO/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_UKMO.netcdf (4.8M)
2021-06-14 09:50:35,206 INFO Download rate 307.8K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/UKMO/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_UKMO.netcdf downloaded OK


attempting to download tprate for GCM METEO_FRANCE, for level surface, year 2021, month 6, in netcdf, attempt 1 of 3



...


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_NCEP.netcdf downloaded OK


attempting to download tprate for GCM JMA, for level surface, year 2021, month 6, in netcdf, attempt 1 of 3



2021-06-14 09:57:00,496 INFO Welcome to the CDS
2021-06-14 09:57:00,498 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-06-14 09:57:00,795 INFO Request is queued
2021-06-14 09:57:52,422 INFO Request is running
2021-06-14 09:58:18,369 INFO Request is completed
2021-06-14 09:58:18,371 INFO Downloading https://download-0003.copernicus-climate.eu/cache-compute-0003/cache/data9/adaptor.mars.external-1623621466.6209579-23100-21-9423f4e0-7b78-4959-9c6d-ae241054778c.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_JMA.netcdf (1M)
2021-06-14 09:58:23,145 INFO Download rate 216.3K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_JMA.netcdf downloaded OK


attempting to download tprate for GCM ECCC, for level surface, year 2021, month 6, in netcdf, attempt 1 of 3



2021-06-14 09:58:23,727 INFO Welcome to the CDS
2021-06-14 09:58:23,729 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-06-14 09:58:24,026 INFO Request is queued
2021-06-14 09:59:15,659 INFO Request is completed
2021-06-14 09:59:15,661 INFO Downloading https://download-0014.copernicus-climate.eu/cache-compute-0014/cache/data7/adaptor.mars.external-1623621549.677284-2531-29-233c832f-ccf7-477c-925c-595fd89d9c01.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_ECCC.netcdf (1.6M)
2021-06-14 09:59:21,547 INFO Download rate 271.9K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC/TPRATE/ensemble_seas_forecasts_tprate_from_2021_06_ECCC.netcdf downloaded OK

